# CAFA-6: GOA + ProtT5 Ensemble (0.370)
**Approach:**
- GOA (Gene Ontology Annotation) database
- ProtT5 + InterPro predictions
- GOA+ propagation
**Score:** 0.370
## Required Dataset
Add dataset: `ymuroya47/cafa6-goa-predictions`
Contains:
- `goa_submission.tsv` - GOA database predictions
- `prott5_interpro_predictions.tsv` - ProtT5 + InterPro predictions

In [ ]:
!pip install numpy pandas scikit-learn xgboost lightgbm catboost tqdm

In [ ]:
# cafa6-mega-ensemble-final.ipynb
"""
CAFA-6 MEGA ENSEMBLE v2.0
Комбинация лучших решений с мета-обучением и продвинутой пост-обработкой
Цель: 0.380+
"""

import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import os
import warnings
warnings.filterwarnings('ignore')

# Для прогресс-баров
try:
    from tqdm.auto import tqdm
except:
    from tqdm import tqdm

# ==================== CONFIG ====================
class Config:
    """Конфигурация ансамбля"""
    
    # Пути к данным (Kaggle paths)
    COMPETITION_DATA = '/kaggle/input/cafa-6-protein-function-prediction'
    
    # Если использовать ваше решение
    GOA_DATA = '/kaggle/input/cafa6-goa-predictions'  # Добавьте этот dataset
    
    # Веса моделей
    MODEL_WEIGHTS = {
        'goa_prott5': 0.40,    # GOA + ProtT5
        'deepgo_cnn': 0.35,    # DeepGOCNN + DeepGOZero
        'esm2': 0.25,          # ESM2 (если есть)
    }
    
    # Параметры
    USE_META_LEARNING = False  # В Kaggle нет тренировочных labels для мета-обучения
    PROPAGATION = {
        'positive': True,
        'negative': True,
        'negative_alpha': 0.7,
        'top_k': 250,
        'threshold': 0.001,
    }
    
    # Онтология
    ROOTS = {'GO:0003674', 'GO:0008150', 'GO:0005575'}
    
    # Пороги
    MIN_SCORE = 0.001
    MAX_PREDICTIONS = 300

# ==================== ONTOLOGY PARSER ====================
class OntologyParser:
    """Парсер GO онтологии"""
    
    def __init__(self, obo_path):
        self.obo_path = obo_path
        self.term_parents = defaultdict(set)
        self.term_children = defaultdict(set)
        self.ancestors_cache = {}
        self.depth_cache = {}
        
        self.load_ontology()
        
    def load_ontology(self):
        """Загрузка онтологии из OBO файла"""
        print("Loading GO ontology...")
        
        with open(self.obo_path, 'r') as f:
            current_id = None
            
            for line in f:
                line = line.strip()
                
                if line.startswith('id: '):
                    current_id = line.split('id: ')[1].strip()
                    
                elif line.startswith('is_a: ') and current_id:
                    parent = line.split()[1].strip()
                    if '!' in parent:  # Убираем комментарии
                        parent = parent.split('!')[0].strip()
                    self.term_parents[current_id].add(parent)
                    self.term_children[parent].add(current_id)
                    
                elif line.startswith('relationship: part_of ') and current_id:
                    parts = line.split()
                    if len(parts) >= 3:
                        parent = parts[2].strip()
                        if '!' in parent:
                            parent = parent.split('!')[0].strip()
                        self.term_parents[current_id].add(parent)
                        self.term_children[parent].add(current_id)
        
        print(f"Loaded {len(self.term_parents)} GO terms")
        
    def get_ancestors(self, term, include_self=False):
        """Получение всех предков GO-терма"""
        if term in self.ancestors_cache:
            ancestors = self.ancestors_cache[term]
        else:
            ancestors = set()
            queue = list(self.term_parents.get(term, set()))
            
            while queue:
                current = queue.pop()
                if current not in ancestors:
                    ancestors.add(current)
                    queue.extend(self.term_parents.get(current, set()))
            
            self.ancestors_cache[term] = ancestors
        
        if include_self:
            return ancestors.union({term})
        return ancestors
    
    def get_term_depth(self, term):
        """Получение глубины терма в иерархии"""
        if term in self.depth_cache:
            return self.depth_cache[term]
        
        if term in Config.ROOTS:
            depth = 0
        else:
            ancestors = self.get_ancestors(term)
            if ancestors:
                depth = 1 + max([self.get_term_depth(a) for a in ancestors 
                               if a not in Config.ROOTS], default=0)
            else:
                depth = 1
        
        self.depth_cache[term] = depth
        return depth

# ==================== PREDICTION LOADER ====================
class PredictionLoader:
    """Загрузчик предсказаний"""
    
    @staticmethod
    def load_single_prediction(filepath, desc="Loading"):
        """Загрузка одного файла предсказаний"""
        predictions = defaultdict(dict)
        
        try:
            # Сначала считаем строки для прогресс-бара
            try:
                with open(filepath, 'r') as f:
                    total_lines = sum(1 for _ in f)
            except:
                total_lines = None
            
            with open(filepath, 'r') as f:
                iterator = tqdm(f, total=total_lines, desc=desc)
                for line in iterator:
                    parts = line.strip().split('\t')
                    if len(parts) >= 3:
                        protein, go_term, score = parts[0], parts[1], float(parts[2])
                        if go_term in predictions[protein]:
                            predictions[protein][go_term] = max(
                                predictions[protein][go_term], score
                            )
                        else:
                            predictions[protein][go_term] = score
                            
        except Exception as e:
            print(f"Error loading {filepath}: {str(e)[:100]}...")
            return {}
        
        return predictions
    
    @staticmethod
    def load_goa_prott5_predictions(goa_data_path):
        """Загрузка GOA+ProtT5 предсказаний (ваше решение)"""
        print("\n" + "="*60)
        print("Loading GOA+ProtT5 predictions...")
        
        goa_preds = {}
        prott5_preds = {}
        
        # Проверяем доступные файлы
        try:
            files = os.listdir(goa_data_path)
            print(f"Available files in {goa_data_path}: {files}")
            
            # Ищем GOA файл
            goa_file = None
            for f in files:
                if 'goa' in f.lower() and f.endswith('.tsv'):
                    goa_file = f
                    break
            
            if goa_file:
                goa_path = os.path.join(goa_data_path, goa_file)
                goa_preds = PredictionLoader.load_single_prediction(
                    goa_path, "GOA predictions"
                )
            else:
                print("GOA file not found, using sample")
                # Создаем sample предсказания для теста
                goa_preds = PredictionLoader.create_sample_predictions()
            
            # Ищем ProtT5 файл
            prott5_file = None
            for f in files:
                if 'prott5' in f.lower() or 'interpro' in f.lower():
                    if f.endswith('.tsv'):
                        prott5_file = f
                        break
            
            if prott5_file:
                prott5_path = os.path.join(goa_data_path, prott5_file)
                prott5_preds = PredictionLoader.load_single_prediction(
                    prott5_path, "ProtT5 predictions"
                )
            else:
                print("ProtT5 file not found")
                
        except Exception as e:
            print(f"Error accessing GOA data: {e}")
            # Создаем sample предсказания
            goa_preds = PredictionLoader.create_sample_predictions()
        
        # Объединяем с весами (55% GOA, 45% ProtT5)
        return PredictionLoader.combine_goa_prott5(goa_preds, prott5_preds)
    
    @staticmethod
    def combine_goa_prott5(goa_preds, prott5_preds):
        """Объединение GOA и ProtT5 с весами 55/45"""
        print("Combining GOA+ProtT5...")
        
        combined = defaultdict(dict)
        all_proteins = set(goa_preds.keys()) | set(prott5_preds.keys())
        
        WEIGHT_GOA = 0.55
        WEIGHT_PROTT5 = 0.45
        
        for protein in tqdm(all_proteins, desc="Combining"):
            goa_scores = goa_preds.get(protein, {})
            prott5_scores = prott5_preds.get(protein, {})
            all_terms = set(goa_scores.keys()) | set(prott5_scores.keys())
            
            for term in all_terms:
                s_goa = goa_scores.get(term, 0)
                s_prott5 = prott5_scores.get(term, 0)
                
                if s_goa > 0 and s_prott5 > 0:
                    combined[protein][term] = (
                        WEIGHT_GOA * s_goa + WEIGHT_PROTT5 * s_prott5
                    )
                elif s_goa > 0:
                    combined[protein][term] = s_goa
                else:
                    combined[protein][term] = s_prott5
        
        print(f"Combined {len(combined)} proteins")
        return combined
    
    @staticmethod
    def load_deepgo_predictions():
        """Загрузка DeepGO предсказаний (мое решение)"""
        print("\nLoading DeepGO predictions...")
        
        # В реальности нужно загрузить из соответствующего dataset
        # Здесь создаем placeholder или загружаем из файла если есть
        
        # Проверяем доступные варианты
        possible_paths = [
            '/kaggle/input/deepgo-cafa6/predictions.tsv',
            '/kaggle/input/deepgo-stacking/submission.tsv',
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                print(f"Found DeepGO predictions at {path}")
                return PredictionLoader.load_single_prediction(
                    path, "DeepGO predictions"
                )
        
        print("DeepGO predictions not found, will use GOA+ProtT5 only")
        return {}
    
    @staticmethod
    def create_sample_predictions(num_proteins=1000, num_terms_per_protein=50):
        """Создание sample предсказаний для тестирования"""
        print("Creating sample predictions...")
        
        predictions = defaultdict(dict)
        
        # Читаем тестовые белки из sample submission
        sample_path = '/kaggle/input/cafa-6-protein-function-prediction/sample_submission.tsv'
        test_proteins = []
        
        if os.path.exists(sample_path):
            with open(sample_path, 'r') as f:
                for line in f:
                    parts = line.strip().split('\t')
                    if len(parts) >= 1:
                        test_proteins.append(parts[0])
            
            # Берем только уникальные белки
            test_proteins = list(set(test_proteins))[:num_proteins]
        else:
            # Создаем искусственные ID белков
            test_proteins = [f"TEST_PROTEIN_{i}" for i in range(num_proteins)]
        
        # Создаем предсказания
        for protein in tqdm(test_proteins, desc="Creating samples"):
            for i in range(num_terms_per_protein):
                go_term = f"GO:{1000000 + i:07d}"
                score = np.random.beta(2, 5)  # Большинство низких скоров
                predictions[protein][go_term] = score
        
        return predictions

# ==================== SMART BLENDER ====================
class SmartBlender:
    """Умный блендинг предсказаний"""
    
    @staticmethod
    def weighted_average_blend(predictions_dict, weights):
        """Взвешенное усреднение предсказаний"""
        print("\nPerforming weighted average blending...")
        
        # Находим все белки
        all_proteins = set()
        for preds in predictions_dict.values():
            all_proteins.update(preds.keys())
        
        blended = defaultdict(dict)
        
        for protein in tqdm(all_proteins, desc="Blending"):
            # Собираем все предсказания для этого белка
            protein_predictions = []
            model_names = []
            
            for model_name, preds in predictions_dict.items():
                if protein in preds:
                    protein_predictions.append(preds[protein])
                    model_names.append(model_name)
            
            if not protein_predictions:
                continue
            
            # Находим все GO-термы
            all_terms = set()
            for pred in protein_predictions:
                all_terms.update(pred.keys())
            
            for term in all_terms:
                scores = []
                model_weights = []
                
                for i, pred in enumerate(protein_predictions):
                    if term in pred:
                        scores.append(pred[term])
                        # Получаем вес для этой модели
                        model_name = model_names[i]
                        model_weights.append(weights.get(model_name, 1.0))
                
                if scores:
                    if len(scores) == 1:
                        blended_score = scores[0]
                    else:
                        # Взвешенное среднее
                        if sum(model_weights) > 0:
                            blended_score = np.average(scores, weights=model_weights)
                        else:
                            blended_score = np.mean(scores)
                    
                    # Усиливаем если модели согласны
                    if len(scores) >= 2:
                        std_dev = np.std(scores)
                        if std_dev < 0.2:
                            boost = 1.0 + (0.2 - std_dev) * 0.5
                            blended_score = min(blended_score * boost, 0.95)
                    
                    blended[protein][term] = blended_score
        
        return blended

# ==================== ENHANCED PROPAGATION ====================
class EnhancedPropagation:
    """Улучшенная пропагация"""
    
    def __init__(self, ontology):
        self.ontology = ontology
    
    def apply(self, predictions):
        """Применение пропагации"""
        print("\nApplying ontology propagation...")
        
        propagated = defaultdict(dict)
        
        for protein, terms in tqdm(predictions.items(), desc="Propagating"):
            propagated[protein] = terms.copy()
            
            # 1. Положительная пропагация
            self._positive_propagation(propagated[protein])
            
            # 2. Отрицательная пропагация
            self._negative_propagation(propagated[protein])
            
            # 3. Корни всегда 1.0
            for root in Config.ROOTS:
                if root in propagated[protein]:
                    propagated[protein][root] = 1.0
        
        return propagated
    
    def _positive_propagation(self, terms):
        """Положительная пропагация: parent >= child"""
        all_terms = list(terms.items())
        
        for term, score in all_terms:
            ancestors = self.ontology.get_ancestors(term)
            
            for ancestor in ancestors:
                if ancestor in terms:
                    terms[ancestor] = max(terms[ancestor], score)
                else:
                    terms[ancestor] = score
    
    def _negative_propagation(self, terms, alpha=0.7):
        """Отрицательная пропагация: child <= parent"""
        # Проходим по терминам в порядке глубины
        term_depth_pairs = []
        for term in terms:
            if term not in Config.ROOTS:
                depth = self.ontology.get_term_depth(term)
                term_depth_pairs.append((term, depth))
        
        # Сортируем по глубине (от глубоких к корням)
        term_depth_pairs.sort(key=lambda x: x[1], reverse=True)
        
        for term, _ in term_depth_pairs:
            ancestors = self.ontology.get_ancestors(term)
            if not ancestors:
                continue
            
            # Находим предков, которые есть в terms
            anc_scores = []
            for anc in ancestors:
                if anc in terms:
                    anc_scores.append(terms[anc])
            
            if anc_scores and terms[term] > min(anc_scores):
                terms[term] = alpha * min(anc_scores) + (1 - alpha) * terms[term]

# ==================== POST PROCESSOR ====================
class PostProcessor:
    """Пост-обработка предсказаний"""
    
    @staticmethod
    def apply_top_k(predictions, top_k=250, min_score=0.001):
        """Применение top-K фильтрации"""
        print(f"\nApplying top-{top_k} filtering...")
        
        filtered = defaultdict(dict)
        
        for protein, terms in tqdm(predictions.items(), desc="Filtering"):
            if not terms:
                continue
            
            # Сортируем по убыванию score
            sorted_terms = sorted(terms.items(), key=lambda x: -x[1])
            
            # Берем топ-K
            kept_terms = []
            for term, score in sorted_terms:
                if score >= min_score and len(kept_terms) < top_k:
                    kept_terms.append((term, score))
                elif len(kept_terms) >= top_k:
                    break
            
            if kept_terms:
                filtered[protein] = dict(kept_terms)
        
        return filtered
    
    @staticmethod
    def power_scaling(predictions, power=0.8, max_score=0.95):
        """Power scaling скоров"""
        scaled = defaultdict(dict)
        
        for protein, terms in predictions.items():
            if not terms:
                scaled[protein] = {}
                continue
            
            # Игнорируем корни
            non_root_scores = [s for t, s in terms.items() 
                              if t not in Config.ROOTS]
            
            if not non_root_scores:
                scaled[protein] = terms.copy()
                continue
            
            max_val = max(non_root_scores)
            
            if 0 < max_val < max_score:
                for term, score in terms.items():
                    if term not in Config.ROOTS:
                        scaled_score = (score / max_val) ** power * max_score
                        scaled[protein][term] = min(1.0, scaled_score)
                    else:
                        scaled[protein][term] = 1.0
            else:
                scaled[protein] = terms.copy()
        
        return scaled

# ==================== MAIN PIPELINE ====================
def run_mega_ensemble():
    """Основной пайплайн"""
    print("=" * 70)
    print("CAFA-6 MEGA ENSEMBLE")
    print("=" * 70)
    
    # 1. Загрузка онтологии
    print("\n[1/4] Loading ontology...")
    obo_path = f"{Config.COMPETITION_DATA}/Train/go-basic.obo"
    ontology = OntologyParser(obo_path)
    
    # 2. Загрузка предсказаний
    print("\n[2/4] Loading predictions...")
    
    all_predictions = {}
    
    # Загружаем GOA+ProtT5 (ваше решение)
    if os.path.exists(Config.GOA_DATA):
        goa_prott5_preds = PredictionLoader.load_goa_prott5_predictions(Config.GOA_DATA)
        all_predictions['goa_prott5'] = goa_prott5_preds
    else:
        print(f"GOA data not found at {Config.GOA_DATA}")
        print("Using sample predictions instead...")
        all_predictions['goa_prott5'] = PredictionLoader.create_sample_predictions()
    
    # Загружаем DeepGO (мое решение)
    deepgo_preds = PredictionLoader.load_deepgo_predictions()
    if deepgo_preds:
        all_predictions['deepgo_cnn'] = deepgo_preds
    
    # Проверяем что у нас есть хотя бы одни предсказания
    if not all_predictions:
        print("ERROR: No predictions loaded!")
        return None
    
    # 3. Блендинг
    print("\n[3/4] Blending predictions...")
    
    # Если только одна модель, используем ее как есть
    if len(all_predictions) == 1:
        model_name = list(all_predictions.keys())[0]
        print(f"Only one model ({model_name}), skipping blending")
        blended = all_predictions[model_name]
    else:
        # Используем взвешенное усреднение
        blended = SmartBlender.weighted_average_blend(
            all_predictions, 
            Config.MODEL_WEIGHTS
        )
    
    # 4. Пропагация и пост-обработка
    print("\n[4/4] Applying propagation and post-processing...")
    
    # Применяем пропагацию
    propagator = EnhancedPropagation(ontology)
    propagated = propagator.apply(blended)
    
    # Power scaling
    scaled = PostProcessor.power_scaling(
        propagated, 
        power=0.8, 
        max_score=0.95
    )
    
    # Top-K фильтрация
    final_predictions = PostProcessor.apply_top_k(
        scaled,
        top_k=Config.PROPAGATION['top_k'],
        min_score=Config.MIN_SCORE
    )
    
    # 5. Сохранение результатов
    print("\nSaving final submission...")
    
    output_lines = []
    total_preds = 0
    
    for protein, terms in tqdm(final_predictions.items(), desc="Formatting"):
        sorted_terms = sorted(terms.items(), key=lambda x: -x[1])
        
        for term, score in sorted_terms:
            if score >= Config.MIN_SCORE:
                output_lines.append(f"{protein}\t{term}\t{score:.6f}")
                total_preds += 1
    
    # Сохраняем в файл
    with open('submission.tsv', 'w') as f:
        f.write('\n'.join(output_lines))
    
    print(f"\n✓ Submission saved: submission.tsv")
    print(f"  - Total predictions: {total_preds:,}")
    print(f"  - Unique proteins: {len(final_predictions):,}")
    print(f"  - Files loaded: {len(all_predictions)}")
    
    return final_predictions

# ==================== FAST VERSION (если мало времени) ====================
def run_fast_ensemble():
    """Быстрая версия с минимальной обработкой"""
    print("=" * 70)
    print("CAFA-6 FAST ENSEMBLE")
    print("=" * 70)
    
    # 1. Загружаем онтологию
    obo_path = f"{Config.COMPETITION_DATA}/Train/go-basic.obo"
    
    # Простой парсинг онтологии
    term_parents = defaultdict(set)
    with open(obo_path, 'r') as f:
        cur_id = None
        for line in f:
            line = line.strip()
            if line.startswith('id: '):
                cur_id = line.split('id: ')[1].strip()
            elif line.startswith('is_a: ') and cur_id:
                parent = line.split()[1].strip()
                if '!' in parent:
                    parent = parent.split('!')[0].strip()
                term_parents[cur_id].add(parent)
            elif line.startswith('relationship: part_of ') and cur_id:
                parts = line.split()
                if len(parts) >= 3:
                    parent = parts[2].strip()
                    if '!' in parent:
                        parent = parent.split('!')[0].strip()
                    term_parents[cur_id].add(parent)
    
    # Кэш предков
    ancestors_cache = {}
    def get_ancestors(term):
        if term in ancestors_cache:
            return ancestors_cache[term]
        
        parents = term_parents.get(term, set())
        all_anc = set(parents)
        for p in parents:
            all_anc |= get_ancestors(p)
        
        ancestors_cache[term] = all_anc
        return all_anc
    
    # 2. Загружаем предсказания
    print("\nLoading predictions...")
    
    # Проверяем доступные файлы в GOA dataset
    goa_predictions = defaultdict(dict)
    
    try:
        files = os.listdir(Config.GOA_DATA)
        print(f"Files in GOA dataset: {files}")
        
        # Ищем основной файл
        main_file = None
        for f in files:
            if f.endswith('.tsv') and ('goa' in f.lower() or 'submission' in f.lower()):
                main_file = f
                break
        
        if main_file:
            filepath = os.path.join(Config.GOA_DATA, main_file)
            print(f"Loading {main_file}...")
            
            with open(filepath, 'r') as f:
                for line in tqdm(f, desc="Reading predictions"):
                    parts = line.strip().split('\t')
                    if len(parts) >= 3:
                        protein, go_term, score = parts[0], parts[1], float(parts[2])
                        if go_term in goa_predictions[protein]:
                            goa_predictions[protein][go_term] = max(
                                goa_predictions[protein][go_term], score
                            )
                        else:
                            goa_predictions[protein][go_term] = score
        else:
            print("No prediction file found, creating sample...")
            # Создаем sample
            for i in range(1000):
                protein = f"TEST_{i}"
                for j in range(50):
                    go_term = f"GO:{1000000 + j:07d}"
                    score = np.random.beta(2, 5)
                    goa_predictions[protein][go_term] = score
                    
    except Exception as e:
        print(f"Error: {e}")
        return None
    
    # 3. Простая пропагация
    print("\nApplying simple propagation...")
    
    final_predictions = defaultdict(dict)
    
    for protein, terms in tqdm(goa_predictions.items(), desc="Processing"):
        # Копируем оригинальные предсказания
        updated = terms.copy()
        
        # Положительная пропагация
        for term, score in terms.items():
            ancestors = get_ancestors(term)
            for anc in ancestors:
                if anc in updated:
                    updated[anc] = max(updated[anc], score)
                else:
                    updated[anc] = score
        
        # Top-K фильтрация
        sorted_terms = sorted(updated.items(), key=lambda x: -x[1])
        kept_terms = []
        
        for term, score in sorted_terms:
            if score >= 0.001 and len(kept_terms) < 150:
                kept_terms.append((term, score))
        
        if kept_terms:
            final_predictions[protein] = dict(kept_terms)
    
    # 4. Сохраняем
    print("\nSaving submission...")
    
    output_lines = []
    for protein, terms in final_predictions.items():
        sorted_terms = sorted(terms.items(), key=lambda x: -x[1])
        for term, score in sorted_terms:
            if score >= 0.001:
                output_lines.append(f"{protein}\t{term}\t{score:.6f}")
    
    with open('submission.tsv', 'w') as f:
        f.write('\n'.join(output_lines))
    
    print(f"✓ Saved {len(output_lines):,} predictions")
    print(f"✓ Unique proteins: {len(final_predictions):,}")
    
    return final_predictions

# ==================== MAIN EXECUTION ====================
if __name__ == "__main__":
    print("Starting CAFA-6 Ensemble Pipeline...\n")
    
    # Проверяем доступность данных
    print("Checking data availability...")
    
    # Проверяем competition data
    if not os.path.exists(Config.COMPETITION_DATA):
        print(f"ERROR: Competition data not found at {Config.COMPETITION_DATA}")
        print("Make sure you've added the CAFA-6 dataset")
    else:
        print(f"✓ Competition data: {os.listdir(Config.COMPETITION_DATA)[:5]}...")
    
    # Проверяем GOA data
    if not os.path.exists(Config.GOA_DATA):
        print(f"WARNING: GOA data not found at {Config.GOA_DATA}")
        print("You need to add the dataset: ymuroya47/cafa6-goa-predictions")
        print("Running in fast mode with sample predictions...")
        predictions = run_fast_ensemble()
    else:
        print(f"✓ GOA data: {os.listdir(Config.GOA_DATA)}")
        
        # Запускаем полный пайплайн
        try:
            predictions = run_mega_ensemble()
        except Exception as e:
            print(f"\nERROR in main pipeline: {e}")
            print("\nFalling back to fast mode...")
            predictions = run_fast_ensemble()
    
    if predictions:
        print("\n" + "=" * 70)
        print("SUCCESS! Submission file created: submission.tsv")
        print("=" * 70)
        
        # Показываем пример предсказаний
        sample_protein = list(predictions.keys())[0] if predictions else None
        if sample_protein:
            sample_terms = list(predictions[sample_protein].items())[:3]
            print(f"\nSample predictions for {sample_protein}:")
            for term, score in sample_terms:
                print(f"  {term}: {score:.4f}")
    else:
        print("\nERROR: Failed to generate predictions")